In [6]:
from dotenv import load_dotenv
import psycopg
import os
import pandas as pd

In [3]:
load_dotenv()

True

In [4]:
# Conexão com o banco de dados
def get_db_connection():
    """Cria e retorna uma conexão com o banco de dados."""
    try:
        conn = psycopg.connect(
            host="localhost",
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            port=os.getenv("DB_PORT")
        )
        return conn
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        raise e

In [8]:
def execute_query(query, params=None):
    """
    Executa uma query SELECT e retorna os resultados como lista de dicionários.
    
    :param conn: conexão do psycopg
    :param query: string com a query SQL
    :param params: parâmetros opcionais para a query (tuple ou list)
    :return: lista de dicionários, onde cada dicionário representa uma linha
    """

    conn = get_db_connection()
    
    with conn.cursor() as cur:
        cur.execute(query, params)
        # Pega os nomes das colunas
        colnames = [desc[0] for desc in cur.description]
        # Pega todas as linhas
        rows = cur.fetchall()
    
    # Cria DataFrame com colunas e dados
    df = pd.DataFrame(rows, columns=colnames)
    return df

In [9]:
def get_month_name(number):
    meses = [
        "",          # índice 0 (vazio, para que 1 = Janeiro)
        "Janeiro",
        "Fevereiro",
        "Março",
        "Abril",
        "Maio",
        "Junho",
        "Julho",
        "Agosto",
        "Setembro",
        "Outubro",
        "Novembro",
        "Dezembro"
    ]

    return meses[number]

In [ ]:
query = (f'''
    select
        dd.month,
        dc.final_classification,
        SUM(f.case_count) AS total_cases
    from
        gold.fact_health_cases f 
    join
        gold.dim_date dd on f.first_symptoms_date_id = dd.date_id
    join
        gold.dim_case_classifications dc ON f.case_classification_id = dc.case_classification_id
    join
        gold.dim_locations dl on f.location_id = dl.location_id
    where
        dc.final_classification <> 'IGNORADO'
    group by 
        dc.final_classification,
        dd.month
    order by
        dd.month;
''')

df = execute_query(query)

df['month_name'] = df['month'].astype(int).apply(get_month_name)

df.head(10)

,month,final_classification,total_cases,month_name
0,1,SRAG NAO ESPECIFICADO,9469,Janeiro
1,1,SRAG POR COVID-19,24561,Janeiro
2,1,SRAG POR INFLUENZA,1262,Janeiro
3,1,SRAG POR OUTRO AGENTE ETIOLOGICO,58,Janeiro
4,1,SRAG POR OUTRO VIRUS RESPIRATORIO,474,Janeiro
5,2,SRAG NAO ESPECIFICADO,6865,Fevereiro
6,2,SRAG POR COVID-19,7100,Fevereiro
7,2,SRAG POR INFLUENZA,65,Fevereiro
8,2,SRAG POR OUTRO AGENTE ETIOLOGICO,53,Fevereiro
9,2,SRAG POR OUTRO VIRUS RESPIRATORIO,548,Fevereiro


In [12]:
query = (f'''
    select
        dd.month,
        dp.pollutant_code,
        avg(f.measurement_value) as monthly_avg_pollution
    from
        gold.fact_air_quality_measurements f 
    join
        gold.dim_date dd on f.date_id = dd.date_id
    join
        gold.dim_pollutants dp on f.pollutant_id = dp.pollutant_id
    join
        gold.dim_locations dl on f.location_id = dl.location_id
    where
        dp.pollutant_code IN ('MP10', 'NO2', 'SO2', 'O3', 'CO', 'MP2,5')
    group by 
        dp.pollutant_code,
        dd.month
    order by
        dd.month;
''')

df = execute_query(query)

df['month_name'] = df['month'].astype(int).apply(get_month_name)

df.head(10)

,month,pollutant_code,monthly_avg_pollution,month_name
0,1,SO2,7.0561400906858114,Janeiro
1,1,MP10,21.9972233336770565,Janeiro
2,1,NO2,8.5876908911041778,Janeiro
3,1,CO,0.72418737574552683897,Janeiro
4,1,"MP2,5",13.0019090160721286,Janeiro
5,1,O3,22.5124180753798592,Janeiro
6,2,O3,24.7344237429743278,Fevereiro
7,2,"MP2,5",12.6436532894959835,Fevereiro
8,2,SO2,6.2652608439415370,Fevereiro
9,2,CO,0.66538828983170412174,Fevereiro
